In [ ]:
import torch
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = "retina"

torch.manual_seed(2023)
np.random.seed(2023)
torch.set_float32_matmul_precision("medium")

In [ ]:
from typing import Any, Tuple

from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

train_dataset = MNIST(download=True, train=True)
val_dataset = MNIST(train=False)


def collate(batch: Any) -> torch.Tensor:
    images = list(zip(*batch))[0]
    image_arrays = np.array([np.array(im).flatten() for im in images], dtype=np.float32)

    return torch.from_numpy(image_arrays / 255.0)


train_loader = DataLoader(
    train_dataset, shuffle=True, batch_size=256, collate_fn=collate
)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=256, collate_fn=collate)

In [ ]:
from lightning.pytorch.utilities.types import STEP_OUTPUT
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import lightning.pytorch as L

import torchvision as tv
from einops import rearrange


class AutoEncoder(L.LightningModule):
    def __init__(self) -> None:
        super().__init__()

        self.encoder = nn.Linear(784, 512)
        self.latent_space = nn.Linear(512, 256)
        self.decoder = nn.Sequential(
            nn.Linear(256, 512), nn.Linear(512, 784), nn.LeakyReLU()
        )

        self.criterion = nn.MSELoss()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.encoder(x)
        out = F.leaky_relu(out)
        out = self.latent_space(out)
        out = F.leaky_relu(out)
        out = self.decoder(out)
        out = F.leaky_relu(out)

        return out

    def configure_optimizers(self) -> Any:
        return optim.AdamW(params=self.parameters(), lr=0.0005)

    def __compute_loss(self, batch: torch.Tensor) -> torch.Tensor:
        out = self(batch)
        loss = self.criterion(out, batch)
        return loss

    def training_step(self, batch: torch.Tensor, batch_idx: int) -> STEP_OUTPUT:
        loss = self.__compute_loss(batch)
        self.log("training_loss", loss)

        return {"loss": loss, "log": {"training_loss": loss}}

    def validation_step(self, batch: torch.Tensor, batch_idx: int) -> STEP_OUTPUT:
        loss = self.__compute_loss(batch)
        self.log("validation_loss", loss)

        # log images
        generated = self(batch)
        # reshape to original size
        generated = torch.unflatten(generated, dim=-1, sizes=(28, 28))
        actuals = torch.unflatten(batch, dim=-1, sizes=(28, 28))

        # create grids
        gen_grid = tv.utils.make_grid(
            rearrange(generated, "batch h w -> batch 1 h w"), nrow=8
        )
        actual_grid = tv.utils.make_grid(
            rearrange(actuals, "batch h w -> batch 1 h w"), nrow=8
        )

        # add to tensorboard
        self.logger.experiment.add_image(
            "generated_images", gen_grid, self.current_epoch
        )
        self.logger.experiment.add_image(
            "actual_images", actual_grid, self.current_epoch
        )

        return {"loss": loss, "log": {"validation_loss": loss}}

In [ ]:
def train(epochs=5):
    logger = L.loggers.TensorBoardLogger(
        "tb_logs",  # type: ignore
        name="autoencoder_mnist",
        log_graph=True,
    )

    trainer = L.Trainer(
        accelerator="gpu", devices=1, max_epochs=epochs, benchmark=True, logger=logger
    )

    model = AutoEncoder()
    trainer.fit(model, train_loader, val_loader)

    return model

In [ ]:
model = train()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs/

In [ ]:
@torch.no_grad()
def inference(image: Any, model=model) -> torch.Tensor:
    inputs = torch.from_numpy(np.array(image, dtype=np.float32) / 255.0)
    gen = model(torch.flatten(inputs))

    gen = torch.unflatten(gen, dim=-1, sizes=(28, 28))

    return gen


sample = val_dataset[3]
gen = inference(sample[0])
plt.imshow(gen.cpu().numpy())
print(sample[1])

In [ ]:
def randomly_pick_n(n: int = 8, max=len(val_dataset)) -> Tuple:
    indexes = torch.randint(0, max, size=(n,)).tolist()
    selection = [val_dataset[i][0] for i in indexes]
    actuals = [torch.from_numpy(np.array(im, dtype=np.float32)) for im in selection]
    generated = [inference(im) for im in selection]

    return torch.stack(generated), torch.stack(actuals)

In [ ]:
generated, actuals = randomly_pick_n()

In [ ]:
def comparison_plot(generated: list, actuals: list) -> None:
    fig = plt.figure(1, figsize=[12, 6])
    fig.tight_layout()

    for idx, gen in enumerate(generated):
        ax = fig.add_subplot(1, len(generated), idx + 1)
        ax.set_title(f"generated_{idx}")
        plt.imshow(gen)
        plt.axis("off")

    for idx, act in enumerate(actuals):
        ax = fig.add_subplot(2, len(actuals), idx + 1)
        ax.set_title(f"actual_{idx}")
        plt.imshow(act)
        plt.axis("off")

    fig.suptitle("Actual (upper) vs Generated(lower)")

In [ ]:
comparison_plot(generated.tolist(), actuals.tolist())